In [1]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os.path
import json
import pickle
import requests

def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 
              'https://www.googleapis.com/auth/script.projects', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file'
              ]
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds

def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 저장된 인증 정보가 없거나, 유효하지 않은 경우 새로운 인증을 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds_filename = 'oauth-new-daebong.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 'token.json'에 저장합니다.
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)

    return creds


In [7]:
def importrangeAllowAccessAutomation(service, get_spreadsheet_id, give_spreadsheet_id, creds):
    url = f'https://docs.google.com/spreadsheets/d/{get_spreadsheet_id}/externaldata/addimportrangepermissions?donorDocId={give_spreadsheet_id}&includes_info_params=true&cros_files=false'
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {creds.token}',
    }
    # 데이터를 JSON 형태로 변환하고, POST 요청을 보냅니다.
    response = requests.post(url, headers=headers)
    # 응답 확인
    if 'application/json' in response.headers.get('Content-Type', ''):
        try:
            data = response.json()
            print(data)
        except ValueError:
            print("JSON 디코딩 실패", response.text)
    else:
        print("응답이 JSON 형식이 아닙니다:", response.text)

    

In [8]:
# signup_spreadsheet_id = "1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU"
# targeet_id = ""

#277
# get_spreadsheetId = "1yX6qBTucOtr4li9PDgI-u_QDKRvv3-1RZkaqm_xDRz4"
#276
get_spreadsheetId = "1V0EBc39eBX1ekf8DGFvCiS5SuVnTm6TehpOkhof7Jlo"
give_spreadsheetId = "1lkkPYvkEzPCoKhUf41U7hiosH3fCU4zzKo8JmqXLbTQ"


if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    importrangeAllowAccessAutomation(service_sheets, get_spreadsheetId, give_spreadsheetId, creds)


JSON 디코딩 실패
